In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import pybind11
else:
    try:
        import pybind11
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/pybind11-install.sh" -O "/tmp/pybind11-install.sh" && bash "/tmp/pybind11-install.sh"
        import pybind11  # noqa: F401

In [ ]:
%%file sum.hpp

int sum(int i, int j);

In [ ]:
%%file sum.cpp

int sum(int i, int j) {
    return i + j;
};

In [ ]:
%%file pybind11_sum.cpp
#include <pybind11/pybind11.h>
#include "sum.hpp"

PYBIND11_MODULE(pybind11_sum, m) {
    m.def("sum", &sum);
}

In [ ]:
%%file setup.py
from distutils.core import Extension, setup

ext_modules = [
    Extension(
        "pybind11_sum", ["sum.cpp", "pybind11_sum.cpp"],
        include_dirs=["/usr/local/pybind11/include"],
        language="c++",
        extra_compile_args=["-std=c++11", "-static-libstdc++"],
        extra_link_args=["-static-libstdc++"]
    ),
]

setup(
    name="pybind11_sum",
    ext_modules=ext_modules
)

In [ ]:
%%bash

CC=gcc CXX=g++ python3 setup.py build_ext -i

In [ ]:
%%bash

LD_PRELOAD="" ldd pybind11_sum*.so | grep libstdc++.so.6 1>/dev/null 2>/dev/null && false || true

In [ ]:
import pybind11_sum
assert pybind11_sum.sum(1, 2) == 3